In [70]:
import numpy as np
import pandas as pd

In [71]:
df = pd.read_csv('C:/Users/Rajvardhan/Desktop/NLP/Fake news classification/fake_job_postings.csv')
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [72]:
df.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [73]:
## Remove the unused columns
col = ['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function']
df.drop(col, axis=1, inplace=True)

In [74]:
df.head()

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0


In [75]:
df.isnull().sum()

description    1
fraudulent     0
dtype: int64

In [76]:
## There is one null value hence dropping the null values
df.dropna(inplace=True)

In [77]:
df.isnull().sum()

description    0
fraudulent     0
dtype: int64

In [78]:
from collections import Counter
from nltk.tokenize import word_tokenize

## Text pre-processing

In [79]:
## Making all the words to lower case
df['description'] = df['description'].str.lower()

In [80]:
import nltk
from nltk.corpus import stopwords
stopwordDictionary = set(stopwords.words('english'))
def Tokenize_data(row):
    review = row
    review = nltk.word_tokenize(review)
    review = [word for word in review if word.isalpha()]
    review = [word for word in review if not word in stopwordDictionary]
    return review

In [81]:
df['processed'] = df['description'].apply(Tokenize_data)

In [82]:
df.head()

,description,fraudulent,processed
0,"food52, a fast-growing, james beard award-winn...",0,"[james, beard, online, food, community, curate..."
1,organised - focused - vibrant - awesome!do you...,0,"[organised, focused, vibrant, awesome, passion..."
2,"our client, located in houston, is actively se...",0,"[client, located, houston, actively, seeking, ..."
3,the company: esri – environmental systems rese...,0,"[company, esri, environmental, systems, resear..."
4,job title: itemization review managerlocation:...,0,"[job, title, itemization, review, managerlocat..."


In [83]:
df.drop('description', axis=1, inplace=True)

In [84]:
df.head()

,fraudulent,processed
0,0,"[james, beard, online, food, community, curate..."
1,0,"[organised, focused, vibrant, awesome, passion..."
2,0,"[client, located, houston, actively, seeking, ..."
3,0,"[company, esri, environmental, systems, resear..."
4,0,"[job, title, itemization, review, managerlocat..."


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17879 entries, 0 to 17879
Data columns (total 2 columns):
fraudulent    17879 non-null int64
processed     17879 non-null object
dtypes: int64(1), object(1)
memory usage: 419.0+ KB


In [86]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

In [87]:
##Y = df['fraudulent']
##X = df['processed']

In [88]:
### Splitting data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(df.processed, df.fraudulent, test_size=0.30, random_state=1, stratify = df.fraudulent)

In [89]:
X_test.head()

9339    [role, developing, exciting, analytics, produc...
7055    [client, seeking, experienced, senior, systems...
8652    [currently, recruiting, exciting, sales, amp, ...
8411    [stokes, brown, toyota, located, beaufort, sc,...
924     [looking, motivated, experienced, ux, professi...
Name: processed, dtype: object

In [90]:
type(Y_test)

pandas.core.series.Series

In [91]:
X_train = X_train.apply(', '.join)
X_test = X_test.apply(', '.join)

In [92]:
max_words = 5000
cv = CountVectoraizer(stop_words='english', max_df=0.95, max_features=max_words, ngram_range=(1,2))

In [93]:
count_train = cv.fit_transform(X_train)
count_test = cv.transform(X_test)

In [94]:
max_words = 1000
tf = TfidfVectorizer(lowercase=True, 
                     stop_words= 'english', 
                     max_df= 0.95, 
                     max_features= max_words)

In [95]:
## X_train = X_train.apply(', '.join)

In [96]:
tf_train = tf.fit_transform(X_train)
tf_test = tf.transform(X_test)

In [97]:
print(count_train)

  (0, 3460)	1
  (0, 2918)	1
  (0, 3570)	1
  (0, 538)	1
  (0, 39)	1
  (0, 3463)	1
  (0, 4845)	1
  (0, 3464)	1
  (0, 4447)	1
  (0, 1694)	1
  (0, 4908)	1
  (0, 2826)	1
  (0, 3468)	1
  (0, 3044)	1
  (0, 4974)	1
  (0, 1697)	1
  (0, 37)	1
  (0, 3067)	1
  (0, 4898)	1
  (0, 4923)	1
  (0, 2582)	1
  (0, 700)	1
  (0, 2655)	1
  (0, 3459)	1
  (0, 38)	1
  :	:
  (12514, 1600)	1
  (12514, 2818)	1
  (12514, 3432)	1
  (12514, 3991)	1
  (12514, 4847)	1
  (12514, 1242)	1
  (12514, 4859)	1
  (12514, 3070)	1
  (12514, 3425)	1
  (12514, 1713)	1
  (12514, 563)	1
  (12514, 3723)	1
  (12514, 1276)	1
  (12514, 2747)	1
  (12514, 3283)	1
  (12514, 3686)	1
  (12514, 3460)	1
  (12514, 4839)	1
  (12514, 1118)	1
  (12514, 1603)	1
  (12514, 4894)	1
  (12514, 3458)	1
  (12514, 1901)	1
  (12514, 2659)	1
  (12514, 1272)	1


In [98]:
pd.DataFrame(count_train.A, columns=cv.get_feature_names()).head()

,aan,abc,abc supply,abilities,ability,ability build,ability communicate,ability demonstrate,ability manage,ability work,...,για,θα,και,με,να,που,την,της,το,του
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Applying random forest on Countvectorizer

In [99]:
from sklearn.ensemble import RandomForestClassifier

In [100]:
rf = RandomForestClassifier()
rf.fit(count_train.A, Y_train)

print('Train accuracy score: ', rf.score(count_train.A, Y_train))
print('Test accuracy score: ', rf.score(count_test.A, Y_test))

Train accuracy score:  0.9997602876548142
Test accuracy score:  0.9778150633855331


### Applying Naive bayes algorithm on Countvectorizer

In [101]:
from sklearn.naive_bayes import MultinomialNB

In [102]:
multiNomialNB = MultinomialNB()
multiNomialNB.fit(count_train.A, Y_train)

print('Train accuracy score: ', multiNomialNB.score(count_train.A, Y_train))
print('Test accuracy score: ', multiNomialNB.score(count_test.A, Y_test))

Train accuracy score:  0.9073911306432281
Test accuracy score:  0.8914988814317674


### Applying random forest using TF-IDF vectorizer

In [103]:
tfRandomForest = RandomForestClassifier()
tfRandomForest.fit(tf_train.A, Y_train)

print('Train accuracy score',  tfRandomForest.score(tf_train.A, Y_train))
print('Test accuracy score', tfRandomForest.score(tf_test.A, Y_test))

Train accuracy score 0.9996803835397523
Test accuracy score 0.9759507829977628


### Applying naive bayes algorithm on TF-IDF vectorizer

In [104]:
multiNomialNBTfIdf = MultinomialNB()
multiNomialNBTfIdf.fit(tf_train.A, Y_train)

print('Train accuracy score: ', multiNomialNBTfIdf.score(tf_train.A, Y_train))
print('Test accuracy score: ', multiNomialNBTfIdf.score(tf_test.A, Y_test))

Train accuracy score:  0.9561326408310028
Test accuracy score:  0.9560029828486204


So the best model is naive bayes using TF-IDF vectorizer as it has good accuracy score and there is no overfit of data